In [ ]:
from PIL import Image

import pytesseract



In [ ]:
# If you don't have tesseract executable in your PATH, include the following:
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'
# Example tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract'


In [ ]:

# Simple image to string
text = pytesseract.image_to_string(Image.open('test.png'), lang= 'eng')


In [ ]:
# # List of available languages
# print(pytesseract.get_languages(config=''))



In [ ]:

# # Batch processing with a single file containing the list of multiple image file paths
# print(pytesseract.image_to_string('images.txt'))

# # Timeout/terminate the tesseract job after a period of time
# try:
#     print(pytesseract.image_to_string('test.jpg', timeout=2)) # Timeout after 2 seconds
#     print(pytesseract.image_to_string('test.jpg', timeout=0.5)) # Timeout after half a second
# except RuntimeError as timeout_error:
#     # Tesseract processing is terminated
#     pass

# # Get bounding box estimates
# print(pytesseract.image_to_boxes(Image.open('test.png')))

# # Get verbose data including boxes, confidences, line and page numbers
# print(pytesseract.image_to_data(Image.open('test.png')))

# # Get information about orientation and script detection
# print(pytesseract.image_to_osd(Image.open('test.png')))


# # Get a searchable PDF
# pdf = pytesseract.image_to_pdf_or_hocr('test.png', extension='pdf')
# with open('test.pdf', 'w+b') as f:
#     f.write(pdf) # pdf type is bytes by default

# # Get HOCR output
# hocr = pytesseract.image_to_pdf_or_hocr('test.png', extension='hocr')

# # Get ALTO XML output
# xml = pytesseract.image_to_alto_xml('test.png')

In [1]:
import os
import csv
from PyPDF2 import PdfReader
import spacy
import re
import yake

In [2]:
def extract_text_from_pdf(pdf_file_path):
    try:
        reader = PdfReader(pdf_file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text
    except Exception as e:
        print("Error:", str(e))
        return None


In [3]:
text = extract_text_from_pdf('/home/supriya/Desktop/FDV/Resume-Parser/data/AI/resume.pdf')

In [4]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\n', ' ', text)

    return text 


In [5]:
preprocessed_text = preprocess_text(text)

### KeyBERT

In [7]:
# from keybert import KeyBERT
# kw_model = KeyBERT()
# keywords = kw_model.extract_keywords(preprocessed_text)
# key_list = keywords(preprocessed_text)
# print(key_list)
# print(keywords)

In [8]:
def extract_skills_keywords(text):
    keyword_extractor = yake.KeywordExtractor()

    skill_keywords_extracted = keyword_extractor.extract_keywords(text)

    skill_keywords = [keyword for keyword, score in skill_keywords_extracted]

    return skill_keywords

In [9]:
extract_skills_keywords(preprocessed_text)

['achievements tasks achievements',
 'tasks achievements tasks',
 'supriya thapa bhujel',
 'data science enthusiast',
 'machine learning deep',
 'learning deep learning',
 'deep steganography learning',
 'deep learning techniques',
 'deep learning president',
 'game deep steganography',
 'deep steganography applying',
 'work achievements tasks',
 'deep learning linux',
 'deep reinforcement learning',
 'reinforcement learning deep',
 'machine learning microdegree',
 'present nepal experienced',
 'ﬂare prediction deep',
 'machine learning model',
 'advanced machine learning']

In [10]:
def extract_skills_from_resume(resume_text):
    preprocessed_text = preprocess_text(resume_text)

    extracted_skills = extract_skills_keywords(preprocessed_text)

    return extracted_skills

In [11]:
skills = extract_skills_from_resume(text)

In [12]:
print(skills)

['achievements tasks achievements', 'tasks achievements tasks', 'supriya thapa bhujel', 'data science enthusiast', 'machine learning deep', 'learning deep learning', 'deep steganography learning', 'deep learning techniques', 'deep learning president', 'game deep steganography', 'deep steganography applying', 'work achievements tasks', 'deep learning linux', 'deep reinforcement learning', 'reinforcement learning deep', 'machine learning microdegree', 'present nepal experienced', 'ﬂare prediction deep', 'machine learning model', 'advanced machine learning']


In [13]:
def extract_education(text):
    education_info = []

    education_pattern = re.compile(
                        r"(?i)(?P<Degree>Bachelor|B\.S\.|Master|M\.S\.|Ph\.D\.|Doctorate).*?"
                        r"(?P<Major>[\w\s]+).*?"
                        r"(?P<University>[\w\s,]+)"
    )

    matches = education_pattern.findall(text)

    for match in matches:
        degree, major, university = match
        education_info.append({
            "Degree": degree.strip(),
            "Major": major.strip(),
            "University": university.strip()
        })

    return education_info


In [14]:
ed_info = extract_education(preprocessed_text)

In [15]:
def format_education_info(education_info):
    formatted_info = []
    for edu in education_info:
        formatted_info.append({
            "Degree": edu["Degree"],
            "Major": edu["Major"],
            "University": edu["University"],
        })
    return formatted_info


In [16]:
format_education_info(ed_info)

[{'Degree': 'bachelor',
  'Major': 'in computer science and information technology  prime college  04 2019   present      nepal  with a focus on data structures  algorithms  physics  math  ai  and oop   skills  python numpy  pandas  scikit learn   keras   sql mysql   tensorﬂow  data visualization  machine learning  deep learning  linux os  leadership  communication  personal projects  digit classiﬁer   09 2022   11 2022    developed the project from scratch using numpy  also did another form of the same project using tensorﬂow  both model achieved over 90  accuracy in classifying digits   solar flare prediction   03 2023   04 2023    developed an advanced machine learning model to accurately forecast solar ﬂares by analyzing diverse solar activity data  demonstrated expertise in data analysis  feature engineering  and model optimization  tictactoe using deep rl   06 2023   07 2023    developed a tic tac toe game using deep reinforcement learning  deep rl  techniques to train ai agents 

In [17]:
def extract_category_from_folder(folder_path):
    return os.path.basename(folder_path)


In [19]:
# extract_category_from_folder('/data')

In [20]:

def extract_top_skills_from_folder(folder_path, top_n):
    top_skills_per_cv = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".pdf"):
                cv_path = os.path.join(root, file)
                resume_text = extract_text_from_pdf(cv_path)
                cleaned_text = preprocess_text(resume_text)
                top_skills = extract_skills_from_resume(cleaned_text)
                top_skills_per_cv.append(top_skills[:top_n])
    return top_skills_per_cv


In [21]:
def extract_education_and_category(folder_path):
    education_and_category_info = []
    category_name = extract_category_from_folder(folder_path)
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".pdf"):
                cv_path = os.path.join(root, file)
                resume_text = extract_text_from_pdf(cv_path)
                cleaned_text = preprocess_text(resume_text)
                education_info = extract_education(cleaned_text)
                education_and_category_info.append({
                    "CV File": cv_path,
                    "Category": category_name,
                    "Education": education_info,
                })
    return education_and_category_info


In [22]:
def save_cv_data_to_csv(category_folder, output_csv_folder, top_n=10):
    top_skills_per_cv = extract_top_skills_from_folder(category_folder, top_n)
    education_and_category_info = extract_education_and_category(category_folder)
    category_name = extract_category_from_folder(category_folder)
    output_csv_path = os.path.join(output_csv_folder, f"{category_name}.csv")

    with open(output_csv_path, 'w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ["CV File", "Category", "Education", "Top Skills"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()

        for item in education_and_category_info:
            cv_path = item["CV File"]
            category = item["Category"]
            education = item["Education"]
            top_skills = top_skills_per_cv.pop(0)
            writer.writerow({
                "CV File": cv_path,
                "Category": category,
                "Education": education,
                "Top Skills": ', '.join(top_skills),
            })


In [23]:

def main():
    DATA_FOLDER = "./data"  # Replace with your folder path
    OUTPUT_CSV_FOLDER = "./output_csv"
    top_n = 10

    if not os.path.exists(OUTPUT_CSV_FOLDER):
        os.makedirs(OUTPUT_CSV_FOLDER)

    for root, dirs, _ in os.walk(DATA_FOLDER):
        for category in dirs:
            category_folder = os.path.join(DATA_FOLDER, category)
            save_cv_data_to_csv(category_folder, OUTPUT_CSV_FOLDER, top_n)

    print(f"CSV files saved to {OUTPUT_CSV_FOLDER}")


In [ ]:

if __name__ == '__main__':
    main()


CSV files saved to ./output_csv


In [ ]:
import pandas as pd

In [ ]:
test_df = pd.read_csv('/home/supriya/Desktop/FDV/Resume-Parser/output_csv/AI.csv')

In [ ]:
test_df.columns

Index(['CV File', 'Category', 'Education', 'Top Skills'], dtype='object')

In [ ]:
test_df['Education'][0]

"[{'Degree': 'bachelor', 'Major': 'in computer science and information\\ntechnology \\nprime college \\n04 2019   present\\n  \\n \\nnepal \\nwith a focus on data\\nstructures  algorithms \\nphysics  math  ai  and oop  \\nskills \\npython numpy  pandas  scikit learn   keras  \\nsql mysql  \\ntensorﬂow \\ndata visualization \\nmachine learning \\ndeep learning \\nlinux os \\nleadership \\ncommunication \\npersonal projects \\ndigit classiﬁer\\n  09 2022   11 2022 \\n \\ndeveloped the project from scratch using numpy \\nalso did another form of the same project using tensorﬂow \\nboth model achieved over 90  accuracy in classifying digits  \\nsolar flare prediction\\n  03 2023   04 2023 \\n \\ndeveloped an advanced machine learning model to accurately forecast\\nsolar ﬂares by analyzing diverse solar activity data  demonstrated\\nexpertise in data analysis  feature engineering  and model optimization \\ntictactoe using deep rl\\n  06 2023   07 2023 \\n \\ndeveloped a tic tac toe game usi

In [ ]:
test_df['Top Skills'][0]

'achievements tasks achievements, tasks achievements tasks, supriya thapa bhujel, data science enthusiast, machine learning deep, learning deep learning, deep steganography learning, deep learning techniques, deep learning president, game deep steganography'